In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
import random

from itertools import product
from datetime import date
from path import Path

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
#     "binarize_params": [6, 12, 25, 50],
#     "binarize_params": [25, 50, 100],
    "binarize_params": [12],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
    "classifier": ["lgbm"],
    "classifier": ["svc"],
#     "classifier": ["knn"],
#     "classifier": ["xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
#     "num_threads": [1], # how many parallel hyperopti 
#     "n_jobs": [1], # how many parallel infer
#     "feat_imp_n_estimators": [1000],
#     "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
#     "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
#     "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
#     "hypers_n_iter": [25],
}

33
c_s 1


In [10]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 12,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm

In [4]:
len(datas)

1

In [5]:
# [x['features'][0] for x in datas]

In [6]:
from mlbt.multiprocess import process_jobs

In [7]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

1

In [8]:
# res = process_jobs(datas, num_threads=num_threads)

In [ ]:
for data in datas:
    run_bt(**data)

2020-02-21 14:22:38,219 config: {'DATA_DIR': Path('C:\\Users\\doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('C:\\Users\\doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'downsampling': 'cusum', 'vol_estimate': 100, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': '

2020-02-21 14:22:44,279 XG#C: Feature engineering for 33 features
2020-02-21 14:22:44,621 Joined 33 features into (71217, 33) shape
2020-02-21 14:22:44,621 EX#C: Feature engineering for 33 features
2020-02-21 14:22:44,963 Joined 33 features into (74018, 33) shape
2020-02-21 14:22:44,964 @YM#C: Feature engineering for 33 features
2020-02-21 14:22:45,288 Joined 33 features into (39311, 33) shape
2020-02-21 14:22:45,289 @BO#C: Feature engineering for 33 features
2020-02-21 14:22:45,640 Joined 33 features into (93715, 33) shape
2020-02-21 14:22:45,641 @C#C: Feature engineering for 33 features
2020-02-21 14:22:45,987 Joined 33 features into (80542, 33) shape
2020-02-21 14:22:45,988 @FV#C: Feature engineering for 33 features
2020-02-21 14:22:46,327 Joined 33 features into (53497, 33) shape
2020-02-21 14:22:46,328 @TY#C: Feature engineering for 33 features
2020-02-21 14:22:46,711 Joined 33 features into (55083, 33) shape
2020-02-21 14:22:46,711 @TU#C: Feature engineering for 33 features
2020-

2020-02-21 14:22:58,015 BD#C: Get bins and feature imps
2020-02-21 14:22:58,031 Dropped label 0.0 0.008053188500795954
2020-02-21 14:22:58,250 bars (68392, 9), events (32049, 2), feats (68392, 33), bins (31779, 3), X_all (29374, 33), X_train (14687, 33)
2020-02-21 14:22:58,255 EZ#C: Get bins and feature imps
2020-02-21 14:22:58,272 Dropped label 0.0 0.03680737551478847
2020-02-21 14:22:58,507 bars (110422, 9), events (42748, 2), feats (110422, 33), bins (41163, 3), X_all (35888, 33), X_train (17944, 33)
2020-02-21 14:22:58,512 XG#C: Get bins and feature imps
2020-02-21 14:22:58,529 Dropped label 0.0 0.0019689599258744498
2020-02-21 14:22:58,752 bars (71217, 9), events (34548, 2), feats (71217, 33), bins (34468, 3), X_all (31713, 33), X_train (15856, 33)
2020-02-21 14:22:58,758 EX#C: Get bins and feature imps
2020-02-21 14:22:58,773 Dropped label 0.0 0.009913905557004957
2020-02-21 14:22:58,998 bars (74018, 9), events (34509, 2), feats (74018, 33), bins (34155, 3), X_all (31715, 33), X_

2020-02-21 14:23:05,505 Dropped label 0.0 0.0031926568891549436
2020-02-21 14:23:05,689 bars (67132, 9), events (10035, 2), feats (67132, 33), bins (9991, 3), X_all (9647, 33), X_train (4823, 33)
2020-02-21 14:23:05,695 QSI#C: Get bins and feature imps
2020-02-21 14:23:05,709 Dropped label 0.0 0.004901960784313725
2020-02-21 14:23:05,912 bars (59640, 9), events (25104, 2), feats (59640, 33), bins (24969, 3), X_all (24533, 33), X_train (12266, 33)
2020-02-21 14:23:05,920 @AD#C [(902, 5), (902, 33), (902,), (903, 5), (903, 33), (903,)]
2020-02-21 14:23:05,921 @BP#C [(2537, 5), (2537, 33), (2537,), (2537, 5), (2537, 33), (2537,)]
2020-02-21 14:23:05,921 @CD#C [(1138, 5), (1138, 33), (1138,), (1139, 5), (1139, 33), (1139,)]
2020-02-21 14:23:05,921 @ED#C [(6605, 5), (6605, 33), (6605,), (6605, 5), (6605, 33), (6605,)]
2020-02-21 14:23:05,922 @ES#C [(12922, 5), (12922, 33), (12922,), (12922, 5), (12922, 33), (12922,)]
2020-02-21 14:23:05,922 @NQ#C [(7133, 5), (7133, 33), (7133,), (7133, 5), 

In [ ]:
%debug